In [1]:
#CELL 1 — Install Dependencies
# This cell installs all required Python packages.
# LangChain and LangChain-community provide the tools and agent framework.
# LangChain-Groq allows us to use Groq LLM.
# Tavily-python and duckduckgo-search provide search capabilities.
!pip install -q langchain langchain-community langchain-groq tavily-python duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [8]:
#CELL 2 — Set API Keys
# In this step, we set environment variables for API keys.
# GROQ_API_KEY is required for Groq LLM.
# TAVILY_API_KEY is required if you want to use Tavily search.
import os

os.environ["GROQ_API_KEY"] = "gsk_"
# Uncomment and set your Tavily API key if available
os.environ["TAVILY_API_KEY"] = "tvly-dev-"


In [9]:
#CELL 3 — Initialize Groq LLM
# In this step, we create an instance of the Groq LLM.
# This LLM will be used later to summarize the results from all tools.
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",  # Using a currently supported model
    temperature=0                   # Temperature 0 for deterministic output
)


In [10]:
#CELL 4 — Define Tools
# In this step, we define three tools for our assistant:
# 1. get_weather - fetches current weather for a city (mock implementation)
# 2. duckduckgo_search - searches top tourist attractions using DuckDuckGo
# 3. tavily_search - searches top tourist attractions using Tavily

from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from tavily import TavilyClient

# DuckDuckGo Tool - no API key required
duckduckgo = DuckDuckGoSearchRun()

@tool
def duckduckgo_search(query: str) -> str:
    """
    Search the web using DuckDuckGo and return results.
    """
    return duckduckgo.run(query)

# Tavily Tool - API key required
tavily_client = TavilyClient()

@tool
def tavily_search(query: str) -> str:
    """
    Search the web using Tavily and return results.
    """
    response = tavily_client.search(query=query, max_results=5)
    return str(response)

# Weather Tool - custom mock tool
@tool
def get_weather(city: str) -> str:
    """
    Fetch current weather for the city.
    This is a mock implementation for demonstration.
    """
    return f"The current weather in {city} is sunny with a temperature of 35°C."


In [11]:
#CELL 5 — Parallel Execution for a Single City
# In this step, we define a function that runs all three tools in parallel
# for a single city. This speeds up the workflow compared to sequential calls.

from concurrent.futures import ThreadPoolExecutor

def fetch_all_parallel(city: str):
    """
    Runs weather, DuckDuckGo search, and Tavily search in parallel for one city.
    Returns a dictionary with results.
    """
    results = {}

    def run_weather():
        results["weather"] = get_weather.run(city)

    def run_ddg():
        results["duckduckgo"] = duckduckgo_search.run(f"top tourist attractions in {city}")

    def run_tavily():
        results["tavily"] = tavily_search.run(f"top tourist attractions in {city}")

    # Execute all three functions in parallel
    with ThreadPoolExecutor() as executor:
        executor.submit(run_weather)
        executor.submit(run_ddg)
        executor.submit(run_tavily)

    return results


In [12]:
#CELL 6 — LLM Summary Function
# In this step, we define a function that takes the results from all tools
# and passes them to the Groq LLM to create a clean, concise summary.

def summarize_results(city: str, results: dict):
    """
    Summarize the weather and top attractions for a city using the LLM.
    """
    prompt = f"""
    You are a travel assistant.

    Destination: {city}

    Weather Information:
    {results['weather']}

    DuckDuckGo Search Results:
    {results['duckduckgo']}

    Tavily Search Results:
    {results['tavily']}

    Please summarize all this information into a concise, user-friendly travel recommendation.
    """
    response = llm.invoke(prompt)
    return response.content


In [13]:
#CELL 7 — Travel Assistant Function for Single City
# This function integrates the parallel execution and summary.
# Input: city name
# Output: final summary string
def travel_assistant(city: str):
    # Step 1: Fetch results from all tools in parallel
    results = fetch_all_parallel(city)

    # Step 2: Generate summary using LLM
    summary = summarize_results(city, results)

    return summary


In [14]:
#CELL 8 — Run Example for One City
# You can pass one city at a time in this cell.
city = "Delhi"
result = travel_assistant(city)

print(f"Travel Assistant Output for {city}:\n")
print(result)


Travel Assistant Output for Delhi:

**Welcome to Delhi!**

**Weather Update:** It's a sunny day in Delhi with a temperature of 35°C. Don't forget to stay hydrated and wear sunscreen.

**Top Tourist Attractions:**

1. **Hanuman Mandir**: An ancient temple with a Guinness Book of Record mention.
2. **Jantar Mantar**: An 18th-century astronomical observatory.
3. **Maharaja Agrasen ki Baoli**: A historic stepwell with intricate carvings.
4. **Kalkaji Temple**: A spiritual sanctuary with deep energy.
5. **Gurudwara Bangla Sahib**: A rejuvenating experience with a soulful atmosphere.
6. **Lodhi Garden**: A beautiful park with monuments from the 15th and 16th centuries.
7. **Chandni Chowk**: India's oldest market, perfect for street food and shopping.
8. **National Museum**: A treasure trove of artifacts and paintings.
9. **Akshardham Temple**: A breathtakingly lavish temple with stunning architecture.
10. **Qutab Minar Complex**: A historic complex with a majestic tower and beautiful gardens

In [15]:
# You can run another city in a separate cell at runtime.
city = "Paris"
result = travel_assistant(city)

print(f"Travel Assistant Output for {city}:\n")
print(result)


Travel Assistant Output for Paris:

**Welcome to Paris, the City of Love!**

**Weather:** Currently sunny with a temperature of 35°C. Don't forget to pack light and stay hydrated.

**Top Attractions:**

1. **Eiffel Tower**: The iconic symbol of Paris, offering breathtaking views of the city.
2. **Louvre Museum**: Home to the Mona Lisa and an extensive collection of art and artifacts.
3. **Notre-Dame Cathedral**: A beautiful and historic church that's a must-visit.
4. **Arc de Triomphe**: A monumental arch that honors the soldiers who fought and died for France.
5. **Montmartre**: A charming neighborhood with narrow streets, charming cafes, and stunning views of the city.
6. **Seine River**: Take a scenic river cruise or stroll along the riverbank to enjoy the city's beauty.
7. **Versailles Palace**: A royal residence with stunning gardens and architecture (best seen on a day trip from Paris).
8. **Sacré-Cœur Basilica**: A beautiful white church perched on a hill, offering stunning view